# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [25]:
from selenium import webdriver #pra controlar o navegador
from selenium.webdriver.common.keys import Keys #controlar teclas do teclado
from selenium.webdriver.common.by import By #pra localizar itens no navegador

navegador = webdriver.Chrome() # variavel pra facilitar usar (acho)

navegador.get('https://www.google.com/') #ja acessa direto, nao precisa botar timer, ele espera carregar
# pegar cotacao dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotacao dolar')#passar qual parametro quer usar e qual o codigo do parametro
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)#importou ali o Keys pra poder usar esse Keys.tecla
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')#atributo dentro la do codigo, de acordo com o que precisar, ai ja colocou na variavel
print(cotacao_dolar)
# pegar cotacao euro
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotacao euro', Keys.ENTER)#aqui dando o enter na mesma linha
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')#atributo dentro la do codigo, de acordo com o que precisar, ai ja colocou na variavel
print(cotacao_euro)
# pegar cotacao ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

navegador.quit()#pra fechar o navegador

#importar base de dados

import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar) #passa linhas e colunas entre virgula [linha, coluna], float pra ja converter ao inserir, vai q nao entra como numero
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#preco de compra = preco original * cotacao
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

#preco de venda = preco de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

5.0129
5.539630467
321.79


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.01290,5012.849871,1.40,7017.989819
1,Carro Renault,4500.00,Euro,5.53963,24928.337102,2.00,49856.674203
2,Notebook Dell,899.99,Dólar,5.01290,4511.559871,1.70,7669.651781
3,IPhone,799.00,Dólar,5.01290,4005.307100,1.70,6809.022070
4,Carro Fiat,3000.00,Euro,5.53963,16618.891401,1.90,31575.893662
5,Celular Xiaomi,480.48,Dólar,5.01290,2408.598192,2.00,4817.196384
6,Joia 20g,20.00,Ouro,321.79000,6435.800000,1.15,7401.170000


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Exportando

In [26]:
tabela.to_excel('ProdutosNovo.xlsx', index=False) #pra nao exportar o indice pra tabela, ele é de uso do python só

- Atualizando os preços e o cálculo do Preço Final

### Agora vamos exportar a nova base de preços atualizada